In [18]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup
import folium as fo


In [3]:
url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Indonesia'
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser") 


In [8]:
table_tbody = soup.find_all("tbody") 
table_td = table_tbody[3].find_all('td')

In [9]:
listkasus = [] 
index = 0
for i in range(34): 
    kasus = table_td[index].text.replace('\n','')
    sembuh = table_td[index+1].text.replace('\n','')
    mati = table_td[index+2].text.replace('\n','') 
    listkasus.append([kasus, sembuh, mati])
    index += 5

In [14]:
table_th = table_tbody[3].find_all('th')
totalkasus = table_th[6].text.replace('\n','')
totalsembuh = table_th[7].text.replace('\n','')
totalmati = table_th[8].text.replace('\n','')
total = [totalkasus, totalsembuh, totalmati] 
total

['1330', '64', '114']

In [29]:
dfprovinsi = pd.read_json('https://raw.githubusercontent.com/LintangWisesa/Indonesia-Covid19-Maps/master/data/gps_indonesia.json')
provinsi = list(dfprovinsi['Provinsi'])
latitude = list(dfprovinsi['latitude'])
longitude = list(dfprovinsi['longitude'])
logo = list(dfprovinsi['logo'])
posisi = [] 
for i in range(len(latitude)): 
    posisi.append([latitude[i], longitude[i]])

In [62]:
peta = fo.Map( 
    location = [-0.7893, 113.9213], 
    zoom_start = 5.2
)

for i in range(len(provinsi)): 
    popup = fo.Popup("Confirmed : {} \nRecovered : {} \nDeaths : {}".format(listkasus[i][0], listkasus[i][1], listkasus[i][2]), max_width=100)
    icon = fo.features.CustomIcon(logo[i], icon_size=(32, 32))
    fo.Marker(
        location = posisi[i],
        popup = popup, 
        tooltip = provinsi[i],
        icon = icon).add_to(peta)
      
    
peta.save('peta.html')